# Markov Chain Monte Carlo (MCMC) Methods: Mathematical Foundations

MCMC methods represent a class of algorithms for sampling from probability distributions where direct sampling is difficult or impossible. These methods construct a Markov chain whose stationary distribution matches the target distribution of interest.

## Core Mathematical Concepts

### 1. Target Distribution

We typically want to sample from a probability distribution with density π(x), where:
- π(x) is known up to a normalizing constant
- π(x) = f(x)/Z where f(x) can be calculated, but Z = ∫f(x)dx is intractable

### 2. Markov Chains

A Markov chain is a sequence of random variables X₀, X₁, X₂, ... such that the conditional distribution of Xₙ₊₁ given X₀, ..., Xₙ depends only on Xₙ:

$$P(X_{n+1} \in A | X_0, X_1, ..., X_n) = P(X_{n+1} \in A | X_n)$$

The transition kernel P(x, y) gives the probability density of moving from state x to state y.

### 3. Stationary Distribution

A distribution π is stationary for a Markov chain if:

$$\pi(y) = \int \pi(x)P(x, y)dx$$

This means that if Xₙ ~ π, then Xₙ₊₁ ~ π as well.

### 4. Detailed Balance

A sufficient (but not necessary) condition for π to be the stationary distribution is detailed balance:

$$\pi(x)P(x, y) = \pi(y)P(y, x)$$

Most MCMC methods are designed to satisfy this condition.

### 5. Ergodicity

A Markov chain is ergodic if it is:
- Irreducible: Can get from any state to any other state with positive probability
- Aperiodic: The chain doesn't get trapped in cycles
- Positive recurrent: Expected return time to any state is finite

For an ergodic chain, the stationary distribution is unique, and the chain will converge to it regardless of the starting point.

## Major MCMC Methods

### 1. Metropolis-Hastings Algorithm

The most fundamental MCMC method with the following steps:

1. Start with an initial state X₀
2. For t = 0, 1, 2, ...:
   - Sample a proposal Y ~ q(·|Xₜ) from a proposal distribution q
   - Calculate the acceptance ratio:
     $$\alpha = \min\left(1, \frac{\pi(Y)q(X_t|Y)}{\pi(X_t)q(Y|X_t)}\right)$$
   - With probability α, set Xₜ₊₁ = Y; otherwise, set Xₜ₊₁ = Xₜ

**Mathematical properties:**
- The transition kernel is: P(x, y) = q(y|x)α(x, y) for x ≠ y
- The detailed balance condition is satisfied: π(x)P(x, y) = π(y)P(y, x)
- The algorithm works even when π is only known up to a normalizing constant

**Special cases:**
- When q is symmetric (q(y|x) = q(x|y)), the acceptance ratio simplifies to min(1, π(Y)/π(Xₜ))
- The original Metropolis algorithm uses a symmetric proposal

### 2. Gibbs Sampling

A special case of Metropolis-Hastings where proposals are always accepted (α = 1).

For a d-dimensional target distribution with x = (x₁, ..., xd):

1. Start with an initial state X₀
2. For t = 0, 1, 2, ...:
   - For i = 1, ..., d:
     - Sample X^(t+1)ᵢ ~ π(·|X^(t+1)₁, ..., X^(t+1)ᵢ₋₁, X^(t)ᵢ₊₁, ..., X^(t)d)
     - (i.e., sample from the conditional distribution of xᵢ given all other variables)

**Mathematical properties:**
- This algorithm samples one component at a time, conditioning on all others
- Proposal distribution q(y|x) places probability 1 on states that differ from x in exactly one component
- Detailed balance is automatically satisfied
- Works well when conditional distributions are easy to sample from

### 3. Slice Sampling

A technique that introduces an auxiliary variable to sample from π(x).

1. Start with an initial state X₀
2. For t = 0, 1, 2, ...:
   - Sample u ~ Uniform(0, π(Xₜ))
   - Define the "slice" S = {x : π(x) ≥ u}
   - Sample Xₜ₊₁ uniformly from S

**Mathematical properties:**
- Transforms the problem into uniform sampling from a (potentially complicated) region
- Joint distribution of (X,u) is uniform over {(x,u) : 0 ≤ u ≤ π(x)}
- Marginal distribution of X is proportional to π(x)
- Detailed balance is satisfied for the joint chain

### 4. Hamiltonian Monte Carlo (HMC)

Uses Hamiltonian dynamics to propose distant states with high acceptance probability.

1. Introduce momentum variables p and define Hamiltonian H(x,p) = U(x) + K(p)
   - U(x) = -log π(x) (potential energy)
   - K(p) = pᵀp/2 (kinetic energy, assuming mass matrix = I)
2. For each iteration:
   - Sample momentum p ~ N(0,I)
   - Simulate Hamiltonian dynamics for L steps using the leapfrog integrator:
     ```
     p' = p - (ε/2)∇U(x)
     x' = x + εp'
     p' = p' - (ε/2)∇U(x')
     ```
   - Accept or reject (x',p') with Metropolis acceptance probability:
     $$\alpha = \min\left(1, \exp(H(x,p) - H(x',p'))\right)$$

**Mathematical properties:**
- The leapfrog integrator is symplectic (preserves volume in phase space)
- Exact Hamiltonian dynamics would preserve H(x,p), giving α = 1
- Numerical integration introduces small errors, requiring the Metropolis correction
- HMC can make large moves in the state space while maintaining high acceptance rates
- Requires gradient information ∇U(x) = -∇log π(x)

### 5. No-U-Turn Sampler (NUTS)

An extension of HMC that automatically tunes the path length parameter L.

1. Build a binary tree of states by integrating both forward and backward in time
2. Double the tree size until a "U-turn" is detected (when further integration starts decreasing the distance between positions)
3. Sample from the set of points in the tree with appropriate probabilities to maintain detailed balance

**Mathematical properties:**
- Eliminates the need to hand-tune the number of leapfrog steps L
- Adapts the trajectory length to the local geometric properties of the target distribution
- Maintains detailed balance despite the adaptive procedure
- Particularly effective for high-dimensional distributions with varying scales

### 6. Metropolis-Adjusted Langevin Algorithm (MALA)

Combines Langevin dynamics with a Metropolis acceptance step.

1. Propose moves using a discretized Langevin diffusion:
   $$Y = X_t + \frac{\epsilon}{2}\nabla \log \pi(X_t) + \sqrt{\epsilon}Z$$
   where Z ~ N(0,I)
2. Accept or reject using Metropolis-Hastings

**Mathematical properties:**
- Uses gradient information to guide proposals toward higher-probability regions
- The proposal distribution is:
  $$q(y|x) = N\left(y; x + \frac{\epsilon}{2}\nabla \log \pi(x), \epsilon I\right)$$
- Less computationally intensive than full HMC but still leverages gradient information
- Efficiency depends critically on the step size ε

## Convergence Diagnostics

### 1. Effective Sample Size (ESS)

Measures the equivalent number of independent samples represented by the MCMC chain:

$$\text{ESS} = \frac{n}{1 + 2\sum_{k=1}^{\infty}\rho_k}$$

where n is the chain length and ρₖ is the autocorrelation at lag k.

### 2. Potential Scale Reduction Factor (PSRF or R̂)

Compares the variance within each chain to the variance between chains:

$$\hat{R} = \sqrt{\frac{W + B/m}{W}}$$

where W is the within-chain variance, B is the between-chain variance, and m is the chain length.

Values of R̂ close to 1 indicate convergence.

## Practical Considerations

### 1. Burn-in Period

Initial samples are influenced by the starting point and are typically discarded:

$$\{X_{b+1}, X_{b+2}, ..., X_n\}$$

where b is the burn-in period.

### 2. Thinning

To reduce autocorrelation, we can keep every kth sample:

$$\{X_k, X_{2k}, ..., X_{\lfloor n/k \rfloor k}\}$$

### 3. Adaptation

Many modern MCMC methods use adaptive schemes to tune parameters during the burn-in phase:
- Adaptive Metropolis adjusts the proposal covariance
- Adaptive HMC tunes both step size ε and mass matrix

## Theoretical Convergence Results

The fundamental convergence theorem for MCMC states that for an ergodic Markov chain with stationary distribution π:

$$\lim_{n \to \infty} \frac{1}{n}\sum_{i=1}^{n}f(X_i) \stackrel{\text{a.s.}}{=} \int f(x)\pi(x)dx$$

for any integrable function f, meaning sample averages converge to expectations under the target distribution.

The convergence rate is typically O(n^(-1/2)), the same as Monte Carlo integration with independent samples, but with a constant that depends on the autocorrelation of the chain.


